In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.datasets import imdb
import keras
from keras.utils import to_categorical
from sklearn.metrics import f1_score

BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
THIRD_MOD_DIR = "/home/abhinav/data/third_model/"
SECOND_MOD_DIR = "/home/abhinav/data/second_model/"
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 200
FOURTH_MOD_DIR = "/home/abhinav/data/fourth_model/"

print('Loading data...')
[x_train, y_train] = pd.read_pickle(FOURTH_MOD_DIR + "train_data_10ratio") 

[x_test, y_test] = pd.read_pickle(FOURTH_MOD_DIR + "test_dataset") 

[x_val, y_val ] = pd.read_pickle(FOURTH_MOD_DIR + "validation_dataset")

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D,merge, MaxPooling1D, Embedding,Merge,  Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers.merge import Add,Concatenate,Dot

num_words = MAX_NB_WORDS

embedding_matrix = pd.read_pickle("/home/abhinav/data/fourth_model/embedding_matrix_for_selected_words")

Using TensorFlow backend.


Loading data...
(13150, 1000) train sequences
(13150, 2) test sequences


In [ ]:
class Metrics(keras.callbacks.Callback):
    def __init__(self,filepath):
        self.filepath = filepath
        self.best = -np.Inf
    def on_epoch_end(self, epoch,batch, logs={}):
        predict = self.model.predict(self.validation_data[0],batch_size = 512)
        predict = prob_to_label(predict)
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        """if self.f1s > self.best:
            self.best = self.f1s
            print('Epoch %05d: improved f1 to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, self.best, self.filepath))
            self.model.save(self.filepath,overwrite =True)"""
        print ("\nWeighted F1 score found on Validation dataset : " ,self.f1s)
        return

from sklearn.metrics import precision_recall_fscore_support
def f1(y_true, y_pred):
    return precision_recall_fscore_support(y_true = y_true, y_pred = y_pred)
ndm = keras.optimizers.Nadam(lr = 0.001)    

def prob_to_label(array):
    for a in array:
        if a[0] > a[1] :
            a[0] = 1
            a[1] = 0
        elif a[0] < a[1]:
            a[0] = 0
            a[1] = 1
        else:
            ind = np.random.randint(2)
            a[ind]= 1
            a[int(1-ind)] = 0
    return array

In [4]:

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=32,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(fsz)(l_conv)
    convs.append(l_pool)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(32, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(10)(l_cov1)
l_flat = Flatten()(l_pool1)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)


model = Model(sequence_input, preds)

model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1000, 200)     10000000    input_2[0][0]                    
____________________________________________________________________________________________________
conv1d_5 (Conv1D)                (None, 998, 32)       19232       embedding_2[0][0]                
____________________________________________________________________________________________________
conv1d_6 (Conv1D)                (None, 997, 32)       25632       embedding_2[0][0]                
___________________________________________________________________________________________

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [5]:
model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

In [6]:
initial_weights = model.get_weights()

In [7]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 5},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13150/13150 [==============================] - 330s - loss: 0.7495 - acc: 0.8684 - val_loss: 0.7727 - val_acc: 0.5916
Epoch 2/10
13150/13150 [==============================] - 323s - loss: 0.3875 - acc: 0.9000 - val_loss: 0.2293 - val_acc: 0.8969
Epoch 3/10
13150/13150 [==============================] - 321s - loss: 0.2219 - acc: 0.9411 - val_loss: 0.1088 - val_acc: 0.9558
Epoch 4/10
13150/13150 [==============================] - 318s - loss: 0.1145 - acc: 0.9739 - val_loss: 0.1361 - val_acc: 0.9586
Epoch 5/10
13150/13150 [==============================] - 323s - loss: 0.0866 - acc: 0.9814 - val_loss: 0.1543 - val_acc: 0.9533
Epoch 6/10
 2048/13150 [===>..........................] - ETA: 153s - loss: 0.0427 - acc: 0.9917

KeyboardInterrupt: 

In [9]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=6,
          class_weight = {0. : 1, 1. : 8},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/6
13150/13150 [==============================] - 330s - loss: 0.9349 - acc: 0.8365 - val_loss: 0.2397 - val_acc: 0.9002
Epoch 2/6
13150/13150 [==============================] - 319s - loss: 0.4858 - acc: 0.8754 - val_loss: 0.3036 - val_acc: 0.8515
Epoch 3/6
13150/13150 [==============================] - 319s - loss: 0.3335 - acc: 0.9151 - val_loss: 0.1961 - val_acc: 0.9131
Epoch 4/6
13150/13150 [==============================] - 318s - loss: 0.2102 - acc: 0.9462 - val_loss: 0.1245 - val_acc: 0.9546
Epoch 5/6
13150/13150 [==============================] - 319s - loss: 0.1417 - acc: 0.9670 - val_loss: 0.2062 - val_acc: 0.9293
Epoch 6/6
13150/13150 [==============================] - 319s - loss: 0.0758 - acc: 0.9834 - val_loss: 0.1321 - val_acc: 0.9639


In [11]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 10},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13150/13150 [==============================] - 331s - loss: 1.1615 - acc: 0.7107 - val_loss: 0.8659 - val_acc: 0.5811
Epoch 2/10
13150/13150 [==============================] - 327s - loss: 0.5823 - acc: 0.8630 - val_loss: 0.3035 - val_acc: 0.8746
Epoch 3/10
13150/13150 [==============================] - 328s - loss: 0.3797 - acc: 0.9068 - val_loss: 0.1738 - val_acc: 0.9302
Epoch 4/10
13150/13150 [==============================] - 336s - loss: 0.2236 - acc: 0.9450 - val_loss: 0.1685 - val_acc: 0.9378
Epoch 5/10
13150/13150 [==============================] - 345s - loss: 0.1243 - acc: 0.9706 - val_loss: 0.1470 - val_acc: 0.9548
Epoch 6/10
13150/13150 [==============================] - 481s - loss: 0.0639 - acc: 0.9858 - val_loss: 0.2071 - val_acc: 0.9426
Epoch 7/10
13150/13150 [==============================] - 640s - loss: 0.0305 - acc: 0.9928 - val_loss: 0.1498 - val_acc: 0.9616
Epoch 8/10
13150/13150 [=======================

In [12]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13150/13150 [==============================] - 693s - loss: 1.0706 - acc: 0.6556 - val_loss: 0.5045 - val_acc: 0.7598
Epoch 2/10
13150/13150 [==============================] - 705s - loss: 0.5904 - acc: 0.8459 - val_loss: 0.2184 - val_acc: 0.9052
Epoch 3/10
13150/13150 [==============================] - 723s - loss: 0.3672 - acc: 0.9135 - val_loss: 0.3231 - val_acc: 0.8786
Epoch 4/10
13150/13150 [==============================] - 713s - loss: 0.2201 - acc: 0.9461 - val_loss: 0.1834 - val_acc: 0.9357
Epoch 5/10
13150/13150 [==============================] - 723s - loss: 0.1254 - acc: 0.9713 - val_loss: 0.1941 - val_acc: 0.9404
Epoch 6/10
13150/13150 [==============================] - 582s - loss: 0.0791 - acc: 0.9821 - val_loss: 0.2769 - val_acc: 0.9314
Epoch 7/10
13150/13150 [==============================] - 392s - loss: 0.0528 - acc: 0.9886 - val_loss: 0.1761 - val_acc: 0.9507
Epoch 8/10
13150/13150 [=======================

In [13]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 15},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13150/13150 [==============================] - 386s - loss: 1.1291 - acc: 0.5487 - val_loss: 0.4567 - val_acc: 0.8166
Epoch 2/10
13150/13150 [==============================] - 383s - loss: 0.6083 - acc: 0.8558 - val_loss: 0.6672 - val_acc: 0.6915
Epoch 3/10
13150/13150 [==============================] - 384s - loss: 0.3382 - acc: 0.9216 - val_loss: 0.2152 - val_acc: 0.9311
Epoch 4/10
13150/13150 [==============================] - 383s - loss: 0.2131 - acc: 0.9534 - val_loss: 0.1476 - val_acc: 0.9494
Epoch 5/10
13150/13150 [==============================] - 383s - loss: 0.1184 - acc: 0.9783 - val_loss: 0.5567 - val_acc: 0.8561
Epoch 6/10
13150/13150 [==============================] - 390s - loss: 0.0739 - acc: 0.9868 - val_loss: 0.1598 - val_acc: 0.9633
Epoch 7/10
13150/13150 [==============================] - 381s - loss: 0.0390 - acc: 0.9933 - val_loss: 0.1779 - val_acc: 0.9621
Epoch 8/10
13150/13150 [=======================

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 20},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13150/13150 [==============================] - 329s - loss: 1.1895 - acc: 0.5576 - val_loss: 0.8259 - val_acc: 0.6510
Epoch 2/10
13150/13150 [==============================] - 327s - loss: 0.6208 - acc: 0.8569 - val_loss: 0.3388 - val_acc: 0.8681
Epoch 3/10
13150/13150 [==============================] - 320s - loss: 0.2989 - acc: 0.9308 - val_loss: 0.2507 - val_acc: 0.9156
Epoch 4/10
13150/13150 [==============================] - 338s - loss: 0.1809 - acc: 0.9594 - val_loss: 0.1472 - val_acc: 0.9574
Epoch 5/10
13150/13150 [==============================] - 349s - loss: 0.1189 - acc: 0.9745 - val_loss: 0.1996 - val_acc: 0.9443
Epoch 6/10
13150/13150 [==============================] - 352s - loss: 0.0667 - acc: 0.9862 - val_loss: 0.1657 - val_acc: 0.9599
Epoch 7/10
13150/13150 [==============================] - 353s - loss: 0.0333 - acc: 0.9931 - val_loss: 0.1570 - val_acc: 0.9646
Epoch 8/10
13150/13150 [=======================

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 30},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13150/13150 [==============================] - 327s - loss: 1.3907 - acc: 0.5082 - val_loss: 0.7245 - val_acc: 0.6907
Epoch 2/10
13150/13150 [==============================] - 358s - loss: 0.7496 - acc: 0.8297 - val_loss: 0.4776 - val_acc: 0.8042
Epoch 3/10
13150/13150 [==============================] - 328s - loss: 0.4249 - acc: 0.9078 - val_loss: 0.2854 - val_acc: 0.8981
Epoch 4/10
10496/13150 [======================>.......] - ETA: 38s - loss: 0.2402 - acc: 0.9487

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 50},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          class_weight = {0. : 1, 1. : 75},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 100},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 125},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 150},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 200},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 500},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 1000},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 150},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 150},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 150},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 150},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [14]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          class_weight = {0. : 1, 1. : 5},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13150/13150 [==============================] - 343s - loss: 0.6347 - acc: 0.8665 - val_loss: 0.2804 - val_acc: 0.8621
Epoch 2/10
13150/13150 [==============================] - 344s - loss: 0.3374 - acc: 0.9134 - val_loss: 0.1784 - val_acc: 0.9226
Epoch 3/10
13150/13150 [==============================] - 344s - loss: 0.1735 - acc: 0.9551 - val_loss: 0.1896 - val_acc: 0.9247
Epoch 4/10
13150/13150 [==============================] - 342s - loss: 0.0699 - acc: 0.9837 - val_loss: 0.1411 - val_acc: 0.9579
Epoch 5/10
13150/13150 [==============================] - 343s - loss: 0.0279 - acc: 0.9938 - val_loss: 0.1258 - val_acc: 0.9633
Epoch 6/10
13150/13150 [==============================] - 344s - loss: 0.0072 - acc: 0.9987 - val_loss: 0.1668 - val_acc: 0.9639
Epoch 7/10
13150/13150 [==============================] - 343s - loss: 0.0017 - acc: 0.9998 - val_loss: 0.2044 - val_acc: 0.9627
Epoch 8/10
13150/13150 [=======================

In [15]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          class_weight = {0. : 1, 1. : 8},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13150/13150 [==============================] - 344s - loss: 0.7176 - acc: 0.8377 - val_loss: 0.3040 - val_acc: 0.8572
Epoch 2/10
13150/13150 [==============================] - 344s - loss: 0.3784 - acc: 0.9021 - val_loss: 0.3005 - val_acc: 0.8628
Epoch 3/10
13150/13150 [==============================] - 344s - loss: 0.1988 - acc: 0.9503 - val_loss: 0.1429 - val_acc: 0.9434
Epoch 4/10
13150/13150 [==============================] - 344s - loss: 0.1038 - acc: 0.9770 - val_loss: 0.1935 - val_acc: 0.9391
Epoch 5/10
13150/13150 [==============================] - 344s - loss: 0.0474 - acc: 0.9891 - val_loss: 0.1875 - val_acc: 0.9498
Epoch 6/10
13150/13150 [==============================] - 345s - loss: 0.0236 - acc: 0.9954 - val_loss: 0.1365 - val_acc: 0.9681
Epoch 7/10
13150/13150 [==============================] - 347s - loss: 0.0095 - acc: 0.9981 - val_loss: 0.2018 - val_acc: 0.9616
Epoch 8/10
13150/13150 [=======================

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=64,
          epochs=6,
          class_weight = {0. : 1, 1. : 10},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          class_weight = {0. : 1, 1. : 15},
validation_data=[x_test, y_test],
         callbacks = [metrics])